## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-30-08-54-00 +0000,wsj,Oil Trades Up Slightly as Traders Watch Geopol...,https://www.wsj.com/finance/commodities-future...
1,2025-12-30-08-51-07 +0000,nyt,"In Lagos, December Is Time to Get Down and ‘De...",https://www.nytimes.com/2025/12/30/world/afric...
2,2025-12-30-08-34-32 +0000,wsj,Stock Market Today: Silver Prices Jump; Dow Fu...,https://www.wsj.com/livecoverage/stock-market-...
3,2025-12-30-08-34-00 +0000,wsj,Euro Holds Steady After Spanish Inflation Data,https://www.wsj.com/finance/stocks/dollar-trad...
4,2025-12-30-08-28-41 +0000,nyt,"In Trump’s D.C. Crackdown, Murder Victims’ Fam...",https://www.nytimes.com/2025/12/29/us/politics...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2356/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
27,trump,35
81,new,18
82,year,14
484,ukraine,13
71,china,12


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
185,2025-12-29-15-53-13 +0000,nypost,Trump scolds Ukraine after Russia claims Putin...,https://nypost.com/2025/12/29/world-news/russi...,78
199,2025-12-29-13-57-13 +0000,nypost,Netanyahu to push back Trump admin’s push for ...,https://nypost.com/2025/12/29/world-news/israe...,76
196,2025-12-29-14-38-40 +0000,nypost,Zelensky splits with Trump and doubts Putin wa...,https://nypost.com/2025/12/29/us-news/volodymy...,74
132,2025-12-29-21-12-47 +0000,nyt,"For Zelensky, Just Keeping Trump Talking About...",https://www.nytimes.com/2025/12/29/world/europ...,74
94,2025-12-29-23-26-00 +0000,wsj,Trump Threatens New Military Action With Israe...,https://www.wsj.com/world/middle-east/trump-to...,74


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
185,78,2025-12-29-15-53-13 +0000,nypost,Trump scolds Ukraine after Russia claims Putin...,https://nypost.com/2025/12/29/world-news/russi...
184,56,2025-12-29-16-06-33 +0000,nypost,China stages military drills around Taiwan to ...,https://nypost.com/2025/12/29/world-news/china...
93,48,2025-12-29-23-27-01 +0000,nyt,Ice Storm Makes Travel Dangerous in Parts of N...,https://www.nytimes.com/2025/12/29/us/new-york...
49,45,2025-12-30-02-43-18 +0000,nypost,Mom discovers her long-lost pen pal is her doc...,https://nypost.com/2025/12/29/us-news/pennsylv...
199,35,2025-12-29-13-57-13 +0000,nypost,Netanyahu to push back Trump admin’s push for ...,https://nypost.com/2025/12/29/world-news/israe...
190,35,2025-12-29-15-07-19 +0000,nypost,Tim Walz bragged about child care business in ...,https://nypost.com/2025/12/29/us-news/tim-walz...
133,33,2025-12-29-21-07-53 +0000,nypost,Dems to convene special hearing on five-year a...,https://nypost.com/2025/12/29/us-news/dems-to-...
142,29,2025-12-29-20-26-41 +0000,nypost,"Google co-founder, Peter Thiel among billionai...",https://nypost.com/2025/12/29/us-news/larry-pa...
23,28,2025-12-30-05-14-28 +0000,bbc,Bangladesh's first female prime minister Khale...,https://www.bbc.com/news/articles/cm2v256vdkyo...
66,26,2025-12-30-01-33-53 +0000,nypost,Century-old NYC pizzeria Totonno’s seeks new o...,https://nypost.com/2025/12/29/us-news/century-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
